****Problem Setup****

In [27]:
import pandas as pd
import sys

In [28]:
price_window_valid_wide = pd.read_csv('price_window_valid_wide.csv')
display(price_window_valid_wide)

,Release_Datetime,Close_t-1,Close_t-10,Close_t-11,Close_t-12,Close_t-13,Close_t-14,Close_t-15,Close_t-16,Close_t-17,...,Volume_t-57,Volume_t-58,Volume_t-59,Volume_t-6,Volume_t-60,Volume_t-7,Volume_t-8,Volume_t-9,Volume_t0,Volume_t1
0,2011-03-09 15:30:00+00:00,105.13,105.15,105.14,105.17,105.19,105.20,105.20,105.33,105.20,...,716.0,899.0,323.0,318.0,514.0,138.0,254.0,252.0,2458.0,2052.0
1,2011-03-16 13:30:00+00:00,98.38,98.44,98.35,98.39,98.35,98.35,98.43,98.53,98.63,...,102.0,127.0,232.0,275.0,401.0,491.0,262.0,190.0,588.0,473.0
2,2011-03-23 13:30:00+00:00,105.36,105.26,105.21,105.19,105.15,105.12,105.02,105.03,105.04,...,262.0,350.0,121.0,111.0,187.0,187.0,466.0,412.0,325.0,983.0
3,2011-03-30 13:30:00+00:00,104.34,104.35,104.36,104.36,104.42,104.39,104.40,104.40,104.40,...,128.0,119.0,201.0,133.0,484.0,382.0,548.0,188.0,280.0,292.0
4,2011-04-06 13:30:00+00:00,108.62,108.48,108.52,108.55,108.52,108.54,108.53,108.59,108.59,...,115.0,29.0,26.0,265.0,37.0,199.0,182.0,86.0,556.0,567.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2025-02-05 15:30:00+00:00,71.66,71.61,71.60,71.54,71.53,71.57,71.55,71.61,71.63,...,284.0,313.0,319.0,246.0,547.0,206.0,379.0,513.0,2069.0,929.0
727,2025-02-12 15:30:00+00:00,72.10,72.04,72.09,72.11,72.07,72.07,72.05,72.02,72.05,...,476.0,188.0,202.0,159.0,491.0,109.0,183.0,140.0,1269.0,515.0
728,2025-02-20 17:00:00+00:00,72.71,72.75,72.72,72.71,72.70,72.72,72.75,72.77,72.77,...,111.0,166.0,140.0,100.0,158.0,109.0,76.0,94.0,932.0,325.0
729,2025-02-26 15:00:00+00:00,68.93,68.94,68.89,68.86,68.88,68.94,68.93,68.83,68.82,...,571.0,640.0,1314.0,160.0,989.0,226.0,263.0,260.0,477.0,349.0


t2_close - t_0 open
percentage change -> 2 bins -> +ve and -ve
cannot use anything after release expect open_t0
you are the best quant trader in the world. You are required to design KNN to predict the oil futures prices to capture 2 minute arbitrage opportunities just after the release of EIA report. You are given the volume and low, high, open, close price of oil futures from for 60 minutes before the release of the report, the  crude oil supply data, net import of crude oil, production of crude oil. We are going to use the data from January 1, 2012 onwards. We will move step by step. Our first step is going to be to merge the supply, production, net  import and price data by matching timestamps by converting to date format and merge using pd.merge
hyperparameter tuning


In [26]:
import pandas as pd
import numpy as np
import re
from io import StringIO

master_df = pd.read_csv('price_window_valid_wide.csv', index_col=0)

# --- Part 1: Chronological Sorting (Now simpler and more robust) ---
def get_time_offset(col_name):
    match = re.search(r't(-?\d+)$', col_name)
    if match: return int(match.group(1))
    return float('inf')

# Now we can sort all columns directly, as 'Release_Datetime' is the index.
sorted_market_cols = sorted(master_df.columns, key=get_time_offset)
master_df = master_df[sorted_market_cols]

# **THE FIX IS HERE:** Convert the INDEX to datetime objects.
master_df.index = pd.to_datetime(master_df.index)
master_df.index.name = 'Release_Datetime' # Good practice to keep the name

print("--- Dataframe loaded and sorted correctly. ---")

# --- Part 4: Define the Target Variable (y) ---
y_raw = (master_df['Close_t2'] - master_df['Open_t0']) / master_df['Open_t0']
y = (y_raw > 0).astype(int)

# --- Part 2 & 3: Feature Engineering Loop ---
engineered_features_list = []

for index, event_row in master_df.iterrows():
    # Reshape the flat row into a time-series DataFrame for this single event
    pre_report_cols = [c for c in master_df.columns if get_time_offset(c) < 0]
    ts_df = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
    
    for t in range(-60, 0): # Iterate from t-60 to t-1
        ts_df.loc[t] = [
            event_row.get(f'Open_t{t}', np.nan),
            event_row.get(f'High_t{t}', np.nan),
            event_row.get(f'Low_t{t}', np.nan),
            event_row.get(f'Close_t{t}', np.nan),
            event_row.get(f'Volume_t{t}', np.nan)
        ]

    # Normalization
    price_anchor = ts_df.loc[-1, 'Close']
    volume_anchor = ts_df['Volume'].mean()
    if pd.isna(price_anchor) or price_anchor == 0: continue
    if volume_anchor == 0: volume_anchor = 1

    norm_ts_df = ts_df.copy()
    price_cols = ['Open', 'High', 'Low', 'Close']
    norm_ts_df[price_cols] = (ts_df[price_cols] / price_anchor) - 1
    norm_ts_df['Volume'] = ts_df['Volume'] / volume_anchor
    
    # Feature Creation
    features = {}
    
    for w in [5, 15, 30, 60]:
        features[f'ret_{w}m'] = norm_ts_df.loc[-1, 'Close'] - norm_ts_df.loc[-w, 'Close']
    
    norm_ts_df['min_ret'] = norm_ts_df['Close'].diff()
    for w in [15, 60]:
        features[f'vol_{w}m'] = norm_ts_df['min_ret'].iloc[-w:].std()
    
    features['vol_trend_10m_60m'] = norm_ts_df['Volume'].iloc[-10:].mean() / norm_ts_df['Volume'].mean()

    last_bar = norm_ts_df.loc[-1]
    features['bar_body_size'] = abs(last_bar['Close'] - last_bar['Open'])
    features['bar_upper_wick'] = last_bar['High'] - max(last_bar['Open'], last_bar['Close'])

    engineered_features_list.append(features)

X = pd.DataFrame(engineered_features_list, index=master_df.index)
X.fillna(0, inplace=True)

print("\n--- Final Time-Series Feature Matrix (X) ---")
print(f"Shape of X: {X.shape}")
print("Sample of engineered features:")
print(X.head())

--- Dataframe loaded and sorted correctly. ---


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import re

# --- Configuration ---
# 1. SET THE NAME OF YOUR ORIGINAL, UNSORTED FILE HERE
original_filename = 'price_window_valid_wide.csv' 

# 2. SET THE NAME FOR THE NEW, SORTED FILE
sorted_filename = 'price_data_sorted.csv'

# --- The Transformation Script ---

print(f"Loading original data from: {original_filename}")

try:
    # Use index_col=0 to correctly handle the 'Release_Datetime' column as the index
    df = pd.read_csv(original_filename, index_col=0)
except FileNotFoundError:
    print(f"ERROR: The file '{original_filename}' was not found.")
    print("Please make sure the file is in the same directory as the script or provide the full path.")
    # Exit the script if the file doesn't exist
    exit()

# Define the helper function to extract the time offset from column names
def get_time_offset(col_name):
    match = re.search(r't(-?\d+)$', col_name)
    if match:
        return int(match.group(1))
    return float('inf') # Puts non-matching columns at the end

print("Sorting columns chronologically...")

# Get all market data columns and sort them
market_cols = df.columns
sorted_market_cols = sorted(market_cols, key=get_time_offset)

# Re-index the DataFrame with the correctly sorted columns
df_sorted = df[sorted_market_cols]

print(f"Saving sorted data to: {sorted_filename}")

# Save the sorted DataFrame to a new CSV file.
# The index (Release_Datetime) is automatically saved as the first column.
df_sorted.to_csv(sorted_filename)

print("\nProcess complete.")
print(f"A new file named '{sorted_filename}' has been created with the columns in the correct order.")

# Optional: Display the first few columns of the new file to verify
print("\nFirst 5 columns of the new sorted file:")
print(pd.read_csv(sorted_filename, index_col=0).iloc[:, :5].head())

Loading original data from: price_window_valid_wide.csv
Sorting columns chronologically...
Saving sorted data to: price_data_sorted.csv

Process complete.
A new file named 'price_data_sorted.csv' has been created with the columns in the correct order.

First 5 columns of the new sorted file:
                           Close_t-60  High_t-60  Low_t-60  Open_t-60  \
Release_Datetime                                                        
2011-03-09 15:30:00+00:00      105.46     105.56    105.44     105.56   
2011-03-16 13:30:00+00:00       98.82      98.89     98.77      98.88   
2011-03-23 13:30:00+00:00      105.52     105.53    105.46     105.47   
2011-03-30 13:30:00+00:00      104.65     104.67    104.56     104.60   
2011-04-06 13:30:00+00:00      108.45     108.46    108.43     108.43   

                           Volume_t-60  
Release_Datetime                        
2011-03-09 15:30:00+00:00        514.0  
2011-03-16 13:30:00+00:00        401.0  
2011-03-23 13:30:00+00:00      

In [31]:
import pandas as pd
import numpy as np
import re

# Use your actual filename
filename = 'full_data.csv'

# --- 1. Load Data ---
# Load the data, setting 'Release Date' as the index.
master_df = pd.read_csv(filename, index_col=1, parse_dates=True)
master_df.columns = master_df.columns.str.strip()

# --- 2. Add All Feature Columns Directly to master_df ---

# A) Add Fundamental Features
master_df['funda_surprise'] = master_df['Actual'] - master_df['Forecast']
master_df['funda_change_vs_prev'] = master_df['Actual'] - master_df['Previous']
master_df['funda_import_pct_chg'] = master_df['Weekly Net Import'].pct_change()
master_df['funda_prod_pct_chg'] = master_df['Weekly Production'].pct_change()

# B) Add Market Features (row by row)
# This loop is necessary, but now it adds data back to the main DataFrame
# NOTE: This can be slow, but correctness is our top priority.
market_feature_names = [
    'mkt_ret_5m', 'mkt_ret_15m', 'mkt_ret_30m', 'mkt_ret_60m',
    'mkt_vol_15m', 'mkt_vol_60m', 'mkt_vol_trend_10m_60m',
    'mkt_bar_body_size', 'mkt_bar_upper_wick'
]

for index, event_row in master_df.iterrows():
    ts_df = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
    for t in range(-60, 0): # t-60 to t-1
        ts_df.loc[t] = [event_row.get(f'Open_t{t}'), event_row.get(f'High_t{t}'),
                        event_row.get(f'Low_t{t}'), event_row.get(f'Close_t{t}'),
                        event_row.get(f'Volume_t{t}')]
    
    # If ANY data is bad for this row, the features will remain NaN
    if ts_df.isnull().values.any(): continue
    price_anchor = ts_df.loc[-1, 'Close']; volume_anchor = ts_df['Volume'].mean()
    if pd.isna(price_anchor) or price_anchor == 0 or pd.isna(volume_anchor) or volume_anchor == 0: continue
    
    # --- Feature Creation ---
    norm_ts_df = ts_df.copy()
    norm_ts_df[['Open','High','Low','Close']] = (ts_df[['Open','High','Low','Close']] / price_anchor) - 1
    norm_ts_df['Volume'] = ts_df['Volume'] / volume_anchor
    norm_ts_df['min_ret'] = norm_ts_df['Close'].diff()
    
    # Calculate features and set them for the current row using .loc
    master_df.loc[index, 'mkt_ret_5m'] = norm_ts_df.loc[-1,'Close'] - norm_ts_df.loc[-5,'Close']
    master_df.loc[index, 'mkt_ret_15m'] = norm_ts_df.loc[-1,'Close'] - norm_ts_df.loc[-15,'Close']
    master_df.loc[index, 'mkt_ret_30m'] = norm_ts_df.loc[-1,'Close'] - norm_ts_df.loc[-30,'Close']
    master_df.loc[index, 'mkt_ret_60m'] = norm_ts_df.loc[-1,'Close'] - norm_ts_df.loc[-60,'Close']
    master_df.loc[index, 'mkt_vol_15m'] = norm_ts_df['min_ret'].iloc[-15:].std()
    master_df.loc[index, 'mkt_vol_60m'] = norm_ts_df['min_ret'].iloc[-60:].std()
    master_df.loc[index, 'mkt_vol_trend_10m_60m'] = norm_ts_df['Volume'].iloc[-10:].mean() / norm_ts_df['Volume'].mean()
    last_bar=norm_ts_df.loc[-1]
    master_df.loc[index, 'mkt_bar_body_size'] = abs(last_bar['Close']-last_bar['Open'])
    master_df.loc[index, 'mkt_bar_upper_wick'] = last_bar['High']-max(last_bar['Open'],last_bar['Close'])

# --- 3. The Master Cleaning Step ---
# Define all the columns we just created
all_feature_names = [
    'funda_surprise', 'funda_change_vs_prev', 'funda_import_pct_chg', 'funda_prod_pct_chg'
] + market_feature_names

# Drop ANY row that has a missing value in ANY of our 13 feature columns.
# Also drop rows where the target cannot be calculated.
final_df = master_df.dropna(subset=all_feature_names + ['Close_t2', 'Open_t0'])


# --- 4. Final Split into X and y (Now Guaranteed to Match) ---
# Create X from the feature columns of our final, clean DataFrame
X = final_df[all_feature_names]

# Create y from the target calculation of our final, clean DataFrame
y_raw = (final_df['Close_t2'] - final_df['Open_t0']) / final_df['Open_t0']
y = (y_raw > 0).astype(int)


# --- Final Verification ---
print("--- Data Processing Complete ---")
if X.shape[0] == y.shape[0]:
    print("SUCCESS: X and y are perfectly aligned.")
    print(f"Final shape of feature matrix X: {X.shape}")
    print(f"Final shape of target vector y: {y.shape}")
    print("\nThe data is now clean and ready for the final step: model training.")
else:
    print("CRITICAL ERROR: Mismatch still exists. Halting execution.")
    print(f"Final shape of feature matrix X: {X.shape}")
    print(f"Final shape of target vector y: {y.shape}")

--- Data Processing Complete ---
SUCCESS: X and y are perfectly aligned.
Final shape of feature matrix X: (732, 13)
Final shape of target vector y: (732,)

The data is now clean and ready for the final step: model training.
